# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Define the problem ----

In [1]:
# Based on job information, we must predict salaries.  

Write the problem in your own words here

In [8]:
#!/usr/bin/env python3
#import your libraries
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import shuffle

#your info here
__author__ = "Khalid Rahman"
__email__ = "kirahman2@gmail.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [3]:
# 1. COMPLETED create model container, 
# 2. test new features
# 3. tuning model 
# 4. Intend on creating new features and seeing how you can improve the model.
# Later: -store model in file -load model

### ---- 3 Clean the data ----

### ---- 4 Explore the data (EDA) ----

In [52]:
class Data:
    def __init__(self, train_file, train_target_file, test_file, col_cat, col_num, col_id, col_target):
        self.col_cat = col_cat
        self.col_num = col_num
        self.col_id = col_id
        self.col_target = col_target
        self.train_df = self._create_train_df(train_file, train_target_file)
#         self.train_df = self._shuffle_data(self.train_df)
        
        self.test_df = self._create_test_df(test_file)
        
    def _create_train_df(self, train_feature_df, train_target_df):
        train_target = self._load_file(train_target_df)
        train_df = self._load_file(train_feature_df)
        train_df = self._join_df(train_df, train_target, col_id)
#         train_df = self._preprocessing(train_df)
        train_df = self._clean_data(train_df)
        train_df = self._label_encode(train_df, self.col_cat)
        return train_df
    
    def _create_test_df(self, test_file):
        test_df = self._load_file(test_file)
#         test_df = self._preprocessing(test_df)
        test_df = self._label_encode(test_df, col_cat)
        return test_df

    def _shuffle_data(self, train_df):
        train_df = shuffle(train_df, random_state=42)
        return train_df

    def _clean_data(self, df):
        train_df = df
        train_df = train_df[train_df.salary>0]
        return train_df
    
    def _label_encode(self, train_df, col_cat):
        le = LabelEncoder()
        for col in col_cat:
#             le.fit(train_df[col])
#             train_df[col] = le.transform(train_df[col])
            train_df[col] = le.fit_transform(train_df[col])
        return train_df
         
    def _join_df(self, train_df, train_target, col_id):
        return pd.merge(train_df, train_target, on=col_id, how='inner')
    
    def _preprocessing(self, train_df):
        return train_df.drop(['jobId'], axis=1)

    def _load_file(self, file):
        return pd.read_csv(file)
    
train_file = '/Users/krahman/work/salarypredictionportfolio_old/data/train_features.csv'
test_file = '/Users/krahman/work/salarypredictionportfolio_old/data/test_features.csv'
train_target_file = '/Users/krahman/work/salarypredictionportfolio_old/data/train_salaries.csv'
col_cat = ['companyId', 'jobType', 'degree', 'major', 'industry']
col_num = ['milesFromMetropolis', 'yearsExperience']
col_all = col_cat + col_num
col_id = 'jobId'
col_target = 'salary'

data = Data(train_file, train_target_file, test_file, col_cat, col_num, col_id, col_target)
# data.train_df

In [28]:
data.train_df.describe()

,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
count,999995.000000,999995.000000,999995.000000,999995.000000,999995.000000,999995.000000,999995.000000,999995.000000
mean,31.004194,3.505312,2.122665,5.419788,3.000336,11.992407,49.529381,116.062398
std,18.169850,2.291726,1.408911,2.398849,2.000701,7.212390,28.877721,38.717163
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000
25%,15.000000,2.000000,1.000000,4.000000,1.000000,6.000000,25.000000,88.000000
50%,31.000000,4.000000,2.000000,7.000000,3.000000,12.000000,50.000000,114.000000
75%,47.000000,6.000000,3.000000,7.000000,5.000000,18.000000,75.000000,141.000000
max,62.000000,7.000000,4.000000,8.000000,6.000000,24.000000,99.000000,301.000000


In [53]:
class FeatureEngineering:
    def __init__(self, data):
        self.data = data
        self.group_df = self.data.train_df.groupby(col_cat)
        self.group_df = self._calculate_stats(self.group_df)
        self.data.train_df = self._merge_df(data.train_df, self.group_df)
        
    def _calculate_stats(self, group_df):
        group_df = pd.DataFrame({'group_mean': group_df[data.col_target].mean()})
        group_df['group_min'] = self.group_df[data.col_target].min()
        group_df['group_max'] = self.group_df[data.col_target].max()
        group_df['group_std'] = self.group_df[data.col_target].std()
        group_df['group_median'] = self.group_df[data.col_target].median()
        return group_df
    
    def _merge_df(self, train_df, group_df):
        final_df = pd.merge(left=train_df, right=group_df, how='left', on=col_cat)
        final_df = final_df.fillna(0)
        return final_df

fe = FeatureEngineering(data)
# fe.group_df

In [31]:
# df_temp = fe.group_df
# df_temp[df_temp['group_mean']==130.875000].shape
# df_temp

In [43]:
# sns.distplot(data.train_df[data.train_df.jobType==0].salary)
# years experience? how does years experience play into this? we see a CEO paid  50k and a janitor
# sns.distplot(data.train_df[data.train_df.jobType==3].salary)
# sns.scatterplot(x = data.train_df[data.train_df.jobType==3].salary, y = data.train_df[data.train_df.jobType==3].yearsExperience, hue=data.train_df.milesFromMetropolis)
# print(data.train_df[data.train_df.jobType==3].salary.max())
# train_df2 = pd.read_csv('/Users/krahman/work/salarypredictionportfolio_old/data/train_features.csv')
# train_target2 = pd.read_csv('/Users/krahman/work/salarypredictionportfolio_old/data/train_salaries.csv')
# df1 = pd.concat([train_df2,train_target2],axis=1)
# df1[df1['jobType']=='JANITOR'].salary.max()
# sns.boxplot(df1[df1['jobType']=='JANITOR'].salary)

In [56]:
class Model:
    def __init__(self, train_features, train_target, model_container, set_cv, num_proc, dir_pred):
        self.train_features = train_features
        self.train_target = train_target
    
    def add_model(self, model):
        model_container[model] = self._score_model(model, self.train_features, self.train_target, set_cv, num_proc)
    
    def fit_model(self, model, train_df, train_target):
        return model.fit(train_df, train_target)
        
    def predict_model(self, model_selected, test_df, train_df, train_target):
        model = self.fit_model(model_selected, train_df, train_target)
        prediction_results = model.predict(test_df)
        self._save_prediction(prediction_results)
        return prediction_results
        
    def best_scoring_model(self, model_container):
        best_model = min(model_container, key=model_container.get)
        best_score = model_container.get(best_model)
        self._print_summary(best_model, best_score)
        return best_model
    
    def _print_summary(self, best_model, best_score):
        print('The best model is:\n', best_model)
        print('\nThe best score is:', best_score)
        
    def _score_model(self, model, train_df, target_train, set_cv, num_proc):
        return -1.0*np.mean(cross_val_score(model, train_df, target_train, scoring='neg_mean_squared_error', cv=set_cv, n_jobs=num_proc))

    def _save_prediction(self, prediction_results):
        prediction_results = pd.DataFrame(prediction_results).to_csv(dir_pred)

# train_features_df = data.train_df
# train_target_df = train_features_df.pop('salary')

# train_features = data.train_df.loc[:1000,:]
# train_target = train_features.pop(col_target)
# train_features = train_features[col_all]

# train_features = fe.group_df.loc[:1000,:]
train_features = data.train_df
train_features = train_features[['companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis', 'group_mean', 'group_min',
       'group_max', 'group_std', 'group_median', 'salary']]
train_target = train_features.pop(col_target)

# train_features = train_features[col_all]
# NEXT, MAKE SURE YOU ARE PASSING THE NEW FEATURES INTO THE MODEL!! 

test_features_df = data.test_df
test_features_df = test_features_df[col_all]

# fe = FeatureEngineering(data)

set_cv = 2
num_proc = -1
dir_pred = '/Users/krahman/work/salarypredictionportfolio_old/data/prediction/prediction_results.csv'
model_container = {}
model = Model(train_features, train_target, model_container, set_cv, num_proc, dir_pred)

model.add_model(GradientBoostingRegressor(loss='ls', learning_rate=.1, alpha=.1, n_estimators=180, subsample=1, max_depth=4))
# model.add_model(GradientBoostingRegressor(n_estimators=40, max_depth=7, loss='ls'))
model.add_model(LinearRegression())
selected_model = model.best_scoring_model(model_container)
model.predict_model(selected_model, test_features_df, train_features_df, train_target_df)

# fe.group_df

The best model is:
 GradientBoostingRegressor(alpha=0.1, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=180,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

The best score is: 314.80121606647765


NameError: name 'train_features_df' is not defined

In [ ]:
# data.train_df

In [67]:
# we need to ingest a model, ingest parameters, create param_grid
class RandomSearch:
    def __init__(self, model):
        self.model = model
    return
gs = RandomSearch()

SyntaxError: 'return' outside function (<ipython-input-67-08502efc1610>, line 5)

In [ ]:
# # TESTING BY INDUSTRY 
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import RandomizedSearchCV
# import time

# # before we score it, we need to predict, then inverse transform
# def scoring_gbr_gs():
#     score_total = 0
#     loss_types = ['ls', 'lad', 'huber', 'quantile']
#     learning_rate = [.001,.01,.1,.2,.3,.4,.5,.6,.7,.8,.9]
#     alpha= [.001,.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,.95]
#     n_estimators = [100,200,300,400,600,1000,2000]
#     subsample =[1,.9,.8,.7,.6,.5,.4,.3,.2,.1]
#     max_depth = [1,2,3,4,5,6,7,8,9,10]
#     model_gbr = GradientBoostingRegressor()
#     param_grid = dict(
#                      loss=['ls'],
#                      learning_rate=[.1],
#                       alpha=[.1],
#                       n_estimators=[180],
#                      subsample=[1],
#                      max_depth=[4]
#                      )
#     random = RandomizedSearchCV(estimator=model_gbr,
#                                 param_distributions=param_grid,
#                                 scoring='neg_mean_squared_error',
#                                 n_jobs=-1,
#                                 cv=5)
    
# #     for val in ['HEALTH', 'WEB', 'AUTO', 'FINANCE', 'EDUCATION', 'OIL', 'SERVICE']:
#     df = data.train_df
#     train_temp = df[df['industry']=='HEALTH']
#     y_train_temp = train_temp.pop('salary')
#     train_temp = train_temp.drop(['industry'],axis=1)

# #     train_temp = train_temp.drop(['jobType', 'degree', 'major', 'industry'],axis=1)

#     random_fit = random.fit(train_temp,y_train_temp)
#     scores = random_fit.best_score_
#     print(scores)
#     score_total = score_total + scores
#     print(random_fit.best_params_)
        
# #     score_avg = score_total/7
# #     print('Average score')
# #     print(score_avg)
#     return scores

# start_time = time.time()
# scores = scoring_gbr_gs()
# elapsed_time = time.time() - start_time
# print(elapsed_time)

In [ ]:
# EDA 
# jobType and salary with specific comparison with high school and no degree
df_hs = df[(df.degree=='HIGH_SCHOOL') | (df.degree=='NONE')]
f, ax = plt.subplots(figsize=(12,6.5))
sns.boxplot(x='jobType',y='salary',hue='degree',data=df_hs, palette='Set3')

In [ ]:
# # COMMENT OUT TESTING
# index_salary_null = df[df['salary'].isnull()].index
# df = df.drop(index_salary_null,axis=0)
data.train_df[data.train_df['salary']<51]


In [ ]:
# EDA
# df.isnull().any()
# df.info()
# df.dtypes
# df.shape
# df.describe()
# df.corr()['salary']
# objective is to predict salary. Do we see patterns with salary level and milesFromMetropolis?
# miles_summary = df.groupby('milesFromMetropolis')
# miles_summary.mean()

In [ ]:
# df['jobType_le_sq'] = df['jobType_le']*df['jobType_le']
# df['jobType_le_plus'] = df['jobType_le'] + df['jobType_le']

In [ ]:
# EDA
# Correlation Matrix
# corr = df[['jobType', 'degree', 'major', 'industry', 'yearsExperience', 'milesFromMetropolis', 'salary', 'industry_le']].corr()
corr = df[['jobType', 'degree', 'major', 'industry', 'yearsExperience',
       'milesFromMetropolis', 'salary', 'jobType_le', 'degree_le', 'major_le',
       'industry_le']].corr()
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.title('Heatmap of Correlation Matrix')
df[['jobType', 'degree', 'major', 'industry', 'yearsExperience',
       'milesFromMetropolis', 'salary', 'jobType_le', 'degree_le', 'major_le',
       'industry_le']].corr()['salary']

In [ ]:
# Categorical: jobType (le), degree (le), major (le), industry (le), 
# Continuous: yearsExperience, milesFromMetropolis
# Target: salary
# jobType will show good correlation with salary
# degree will show some correlation with salary
# major will show very little correlation with salary
# industry_le will have salary scattered everywhere. 
# yearsExperience will show correlation with salary
# milesFromMetropolis will show correlation with salary

# EDA
# Boxplot of salary by profession
f, ax = plt.subplots(figsize=(12,6.5))
plt.title("Salary by Job Title")
sns.boxplot(x='jobType',y='salary', data=df, width=.3, color='#eeefff',
            order=['JANITOR','JUNIOR','SENIOR','MANAGER','VICE_PRESIDENT','CFO','CTO','CEO'])
sns.despine(offset=10,trim=True)

In [ ]:
# EDA
# Boxplot of salary by profession
f, ax = plt.subplots(figsize=(9,6.5))
plt.title("Salary by Degree")
sns.boxplot(x='degree',y='salary', data=df, width=.25, color='#eeefff',
            order=['NONE','HIGH_SCHOOL','BACHELORS','MASTERS','DOCTORAL']
            )
sns.despine(offset=9,trim=True)
df['degree'].unique()

In [ ]:
# EDA
# Boxplot of salary by major
f, ax = plt.subplots(figsize=(12,6.5))
plt.title("Salary by Major")
sns.boxplot(x='major',y='salary', data=df, width=.3, color='#eeefff',
            order=['NONE', 'LITERATURE', 'BIOLOGY', 'CHEMISTRY', 'PHYSICS', 'COMPSCI',
            'MATH', 'BUSINESS', 'ENGINEERING'])
sns.despine(offset=10,trim=True)

# # Create mean ascending ranking list for major
# print(df.groupby('major').salary.mean().sort_values().keys())
# df.groupby('major').salary.mean().sort_values()

In [ ]:
# EDA
# Boxplot of salary by major
f, ax = plt.subplots(figsize=(10,6.5))
plt.title("Salary by Industry")
sns.boxplot(x='industry',y='salary', data=df, width=.3, color='#eeefff',
            order=['EDUCATION', 'SERVICE', 'AUTO', 'HEALTH', 'WEB', 'FINANCE', 'OIL']
           )
sns.despine(offset=11,trim=True)

In [ ]:
# # EDA
# # There doesn't appear to be any patterns in salary based on companyId. 
# sns.boxplot(x='companyId',y='salary', data=df_companyid);

In [ ]:
# EDA
# There is no sign that yearsExperience or milesFromMetropolis have differing ranking when boxplotted with 
# 'jobType', 'degree', 'major', 'industry'.

f, axes = plt.subplots(ncols=2, figsize=(25,3))
sns.boxplot(x='jobType', y='yearsExperience', data=df, ax=axes[0]).set_title('Experience Versus Job')
sns.boxplot(x='degree', y='yearsExperience', data=df, ax=axes[1]).set_title('Experience Versus Degree');

In [ ]:
# EDA
f, axes = plt.subplots(ncols=2, figsize=(25,3))
sns.boxplot(x='major', y='yearsExperience', data=df, ax=axes[0]).set_title('Experience Versus Major')
sns.boxplot(x='industry', y='yearsExperience', data=df, ax=axes[1]).set_title('Experience Versus Industry');

In [ ]:
# EDA
f, axes = plt.subplots(ncols=2, figsize=(25,3))
sns.boxplot(x='jobType', y='milesFromMetropolis', data=df, ax=axes[0]).set_title('Distance Versus Job')
sns.boxplot(x='degree', y='milesFromMetropolis', data=df, ax=axes[1]).set_title('Distance Versus Degree');

In [ ]:
# EDA
f, axes = plt.subplots(ncols=2, figsize=(25,3))
sns.boxplot(x='major', y='milesFromMetropolis', data=df, ax=axes[0]).set_title('Distance Versus Major')
sns.boxplot(x='industry', y='milesFromMetropolis', data=df, ax=axes[1]).set_title('Distance Versus Industry');

### ---- 5 Establish a baseline ----

In [ ]:
# we must look very carefully at the data and see opportunities to create more signal in the data that will make 
# the algorithm pick up on certain things that enhance accuracy of predictions. 

### ---- 6 Hypothesize solution ----

In [ ]:
# brainstorm 3 models that you think may improve results over the baseline model based
# read documentation and determine good models to try 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

# FEATURE ENGINEERING NOTES 
# jobType - we've already created this features
# degree - create feature, 0=high school, none, 1=others.
# degree - create another feature, 0=high school, none, 1=bachelors,masters, 2=phd

### ---- 8 Create models ----

In [ ]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [ ]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [ ]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data